In [1]:
!pip install ipywidgets
from ipywidgets import FloatProgress

  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
Using cached fqdn-1.5.1-py3-none-any.whl (9.1 kB)
Using cached isoduration-20.11.0-py3-none-any.whl (11 kB)
Using cached uri_template-1.3.0-py3-none-any.whl (11 kB)


In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 4090 Laptop GPU'

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import time
from tqdm import tqdm


In [37]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Grayscale(num_output_channels=1), # 흑백
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Grayscale(num_output_channels=1), # 흑백
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [38]:
# import os

# acc_directory = r"C:/Users/SKTPJ13/Desktop/Data/엑셀밟음"
# bk_directory = r"C:/Users/SKTPJ13/Desktop/Data/브레이크밟음"

# # 각 경로에 파일이 있는지 확인
# print(os.path.exists(acc_directory))
# print(os.path.exists(bk_directory))


In [39]:
pip install tqdm

In [40]:
import cv2
import os
import random
from tqdm import tqdm

acc_directory = r'C:/Users/SKTPJ13/Desktop/Data0815/acc'
bk_directory = r'C:/Users/SKTPJ13/Desktop/Data0815/brake'
acc_push_directory = r"C:/Users/SKTPJ13/Desktop/Data0815/acc_push"
bk_push_directory = r"C:/Users/SKTPJ13/Desktop/Data0815/brake_push"

acc_directory = sorted([os.path.join(acc_directory, f) for f in os.listdir(acc_directory)])
bk_directory = sorted([os.path.join(bk_directory, f) for f in os.listdir(bk_directory)])
acc_push_directory = sorted([os.path.join(acc_push_directory, f) for f in os.listdir(acc_push_directory)])
bk_push_directory = sorted([os.path.join(bk_push_directory, f) for f in os.listdir(bk_push_directory)])

images_filepaths = [*acc_directory, *bk_directory, *acc_push_directory, *bk_push_directory]

# tqdm으로 진행 상황 표시
correct_images_filepaths = [i for i in tqdm(images_filepaths) if cv2.imread(i) is not None]

random.seed(42)
random.shuffle(correct_images_filepaths)

print(len(correct_images_filepaths))


100%|████████████████████████████████████████████████████████████████████████████| 10259/10259 [03:23<00:00, 50.42it/s]

10259


In [41]:
import cv2
from PIL import Image, ImageFile

# 손상된 이미지도 로드할 수 있도록 설정
ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_images_in_batches(image_paths, batch_size=10):
    correct_images = []
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        for path in batch:
            try:
                # PIL을 사용해 이미지 열기 (손상된 이미지를 걸러내기 위함)
                img = Image.open(path)
                img.verify()  # 이미지를 검증하여 손상 여부 확인
                image = cv2.imread(path)  # OpenCV로 이미지 읽기
                if image is not None:
                    correct_images.append(path)
            except (IOError, SyntaxError, cv2.error) as e:
                print(f"Skipping corrupted image: {path}")
    return correct_images

# images_filepaths는 이미지 파일 경로들의 리스트
correct_images_filepaths = load_images_in_batches(images_filepaths)


In [42]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [43]:
import numpy as np
from sklearn.model_selection import train_test_split

# 예를 들어, correct_images_filepaths는 이미지 파일 경로의 리스트입니다.
#correct_images_filepaths = [...]  # 당신의 이미지 파일 경로 리스트

# 전체 데이터를 학습, 검증, 테스트 세트로 분할합니다.
# 먼저, 학습 세트와 나머지 데이터를 분할합니다.
train_images_filepaths, temp_paths = train_test_split(correct_images_filepaths, test_size=0.4, random_state=42)

# 나머지 데이터(검증 + 테스트)를 검증 세트와 테스트 세트로 분할합니다.
val_images_filepaths, test_images_filepaths = train_test_split(temp_paths, test_size=0.5, random_state=42)

# 이제 train_paths, val_paths, test_paths에 학습, 검증, 테스트 데이터가 나누어졌습니다.


In [44]:

import torch
from torch.utils.data import Dataset
from PIL import Image
class CarDataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)

        label = img_path.split('/')[-1].split('\\')[0]
        #print(label)
        if 'brake_push' in label:
            label = 1
        elif 'acc_push' in label:
            label = 0
        elif 'acc' in label:
            label = 2
        elif 'brake' in label:
            label = 3

        return img_transformed, label

In [45]:
print(len(test_images_filepaths))

2052


In [46]:
size = 224
mean = (0.5,)
std = (0.5,)
batch_size = 32

In [61]:
train_dataset = CarDataset(train_images_filepaths, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = CarDataset(val_images_filepaths, transform=ImageTransform(size, mean, std), phase='val')

index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([1, 224, 224])
1


In [62]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloader_dict = {'train': train_dataloader, 'val': val_dataloader}

batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
# print(inputs.size())
# print(label)

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import time
from tqdm import tqdm


In [64]:
from torchvision import models
from torchvision.models import ResNet18_Weights
import torch.nn as nn

# # Pretrained ResNet18 모델 불러오기
# # pretrained=True 대신 weights=ResNet18_Weights.IMAGENET1K_V1 사용
# model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# # 필요한 경우 마지막 레이어 수정 (예: 클래스 개수가 다른 경우)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 4)  # 4는 클래스 수에 따라 조정

model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# 첫 번째 Conv2d 레이어의 입력 채널을 1로 변경 (흑백 이미지에 맞게)
model.conv1 = nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size, 
                        stride=model.conv1.stride, padding=model.conv1.padding, bias=False)

# 마지막 fully connected 레이어의 출력 뉴런 수를 4로 변경 (4개의 클래스 예측)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)




In [65]:
optimizer = optim.Adam(model.parameters(), lr=1e-7)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [66]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):
    since = time.time()
    best_acc = 0.0

    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(dataloader_dict[phase]):
                #print(inputs,labels)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    pred_prob, preds = torch.max(outputs, 1) # 예측확률, 예측정답
                    # print
                    #print(outputs,labels)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model,best_model_wts

In [ ]:
import time
num_epoch = 100
model = model.to(device)
model,best_model_wts = train_model(model, dataloader_dict, criterion, optimizer, num_epoch)

Epoch 1/100
--------------------


 64%|███████████████████████████████████████████████████▍                            | 124/193 [01:40<00:55,  1.25it/s]

In [ ]:
def test(model, test_images_filepaths):
    id_list = []
    pred_list = []
    _id=0
    with torch.no_grad():
        for test_path in tqdm(test_images_filepaths):
            img = Image.open(test_path)
            _id =test_path.split('/')[-1]
            transform = ImageTransform(size, mean, std)
            img = transform(img, phase='val') #val로 설정하면 랜덤 주지 않음
            img = img.unsqueeze(0) #배치가 1인것을 추가해줌 (토치는 패치단위로 학습하도록 되어있으므로 첫번째에 무조건 배치사이즈가 들어가야함)
            #unsqueeze는 해당자리에 1을 넣어줌 현재는 0에 넣었으므로 맨 앞에 배치사이즈를 추가함
            img = img.to(device)

            model.eval()
            outputs = model(img)
            #preds = F.softmax(outputs, dim=1)[:, 1].tolist()
            preds, pred_ans = torch.max(outputs, 1)
            id_list.append(_id)
            pred_list.append(pred_ans.cpu().item())
    return id_list, pred_list



In [ ]:
# 모델 테스트
#model = ...  # 미리 학습된 모델 로드
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
id_list, pred_list= test(model, test_images_filepaths)

#print(id_list)
#print(pred_list)


In [ ]:
true_labels=[]
for i in id_list:
    if 'brake_push' in i:
      true_labels.append(1)
    elif 'acc_push' in i:
      true_labels.append(0)
    elif 'acc' in i:
      true_labels.append(2)
    elif 'brake' in i:
      true_labels.append(3)


In [ ]:
def calculate_accuracy(true_labels, pred_labels):
    correct = sum(t == p for t, p in zip(true_labels, pred_labels))
    accuracy = correct / len(true_labels)
    return accuracy

In [ ]:
def visualize_predictions(image_paths, true_labels, pred_labels, num_images=5):
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        img = Image.open(image_paths[i])
        true_label = true_labels[i]
        pred_label = pred_labels[i]

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f'True: {true_label}\nPred: {pred_label}')
        plt.axis('off')

    plt.show()

In [ ]:
# 정확도 계산
accuracy = calculate_accuracy(true_labels, pred_list)
print(f'Accuracy: {accuracy:.4f}')

# 예측 시각화
visualize_predictions(test_images_filepaths, true_labels, pred_list)


In [ ]:
import torch

# 예: 모델 객체가 `model`이라고 가정
torch.save(best_model_wts, 'gray_data_0815_left_best_model_wts_18_epoch100.pth')


In [188]:
# import torch
# import torch.nn as nn
# import cv2
# import numpy as np
# from torchvision import models, transforms
# from PIL import Image

# class ImageTransform():
#     def __init__(self, resize, mean, std):
#         self.data_transform = {
#             'train': transforms.Compose([
#                 transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean, std)
#             ]),
#             'val': transforms.Compose([
#                 transforms.Resize(256),
#                 transforms.CenterCrop(resize),
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean, std)
#             ])
#         }

#     def __call__(self, img, phase='val'):
#         return self.data_transform[phase](img)

# # Pretrained ResNet18 모델 로드 및 수정q
# model = models.resnet18(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 4)  # 필요한 경우 클래스 개수로 수정

# # 모델 초기화 및 가중치 로드
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)
# model.load_state_dict(torch.load('best_model_wts_50.pth', map_location=device))
# model.eval()

# # 이미지 전처리 파이프라인
# resize = 224
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
# transform = ImageTransform(resize, mean, std)

# # 클래스 이름 (예: 0: 'class1', 1: 'class2', 2: 'class3', 3: 'class4')
# class_names = ['acc_push', 'brake_push', 'acc', 'brake']  # 실제 클래스 이름으로 교체

# # 웹캠 열기
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     # 프레임 전처리
#     img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#     img = transform(img, phase='val').unsqueeze(0).to(device)

#     # 모델 예측
#     with torch.no_grad():
#         outputs = model(img)
#         _, preds = torch.max(outputs, 1)
#         label = class_names[preds[0]]
        
#         # 예측 결과 확인
#         print(f"Predicted label: {label}")

#     # 결과 표시
#     cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
#     cv2.imshow('Webcam', frame)
    
#     # 'q' 키를 누르면 종료
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()
